In [18]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from keras.utils import np_utils

In [19]:
text = open('shakespere.txt', 'r').read()
text = text[:5000]

In [32]:
# Getting sorted listof char in text
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Spitting text into sentences
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('number of sequences:', len(sentences))

# Character level One - hot Encoding
x_train = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_train[i, t, char_indices[char]] = 1
    y_train[i, char_indices[next_chars[i]]] = 1

total chars:  53
number of sequences: 1654


In [33]:
 # Next Batch Generator
def next_batch(x, y, batch_size):
    N = x.shape[0]
    batch_indices = np.random.permutation(N)[:batch_size]
    x_batch = x[batch_indices]
    y_batch = y[batch_indices]
    return x_batch, y_batch
    

In [34]:
# Data Dimensions
input_dim = len(chars)       # input dimension
seq_max_len = maxlen         # sequence maximum length
out_dim = len(chars)         # output dimension 

In [39]:
# Hyper-Parameters
learning_rate = 0.01    # The optimization initial learning rate
training_steps = 1000  # Total number of training steps
batch_size = 64        # batch size
display_freq = 100    # Frequency of displaying the training results
num_hidden_units = 64   # number of hidden units 

In [40]:
tf.reset_default_graph()

In [41]:
def weight_variable(shape):
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W',
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(shape):
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b',
                           dtype=tf.float32,
                           initializer=initial)

with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32, shape=[None, seq_max_len, input_dim], name='X')
    y = tf.placeholder(tf.float32, shape=[None, out_dim], name='Y')
    keep_prob = tf.placeholder(tf.float32)

In [42]:
def LSTM(x, num_hidden, out_dim, name, use_activation=True, keep_prob = None):
    with tf.variable_scope(name):
        # create weight matrix initialized randomely from N~(0, 0.01)
        weights = weight_variable(shape=[num_hidden_units, out_dim])

        # create bias vector initialized as zero
        biases = bias_variable(shape=[out_dim])

        cell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden)
        outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
        out = tf.matmul(outputs[:, -1, :], weights) + biases
        
        layer = tf.nn.dropout(out, keep_prob)
        
        layer = tf.layers.dense(layer,units=out_dim)
        
        layer = tf.nn.softmax(layer)
        
    
        return layer
    

In [43]:
logits_out = LSTM(x, num_hidden_units, out_dim=out_dim, name = 'lstm1',use_activation=True, keep_prob = 0.2)

W0703 03:22:34.027313 139693720700736 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [44]:
loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, logits_out))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [45]:
# Creating the op for initializing all variables
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(training_steps):
    x_batch, y_batch = next_batch(x_train, y_train, batch_size)
    _, batch_loss = sess.run([train_op, loss], feed_dict={x: x_batch, y: y_batch})
    if i % display_freq == 0:
        print('Step {}, Loss={}'.format(i, batch_loss))

Step 0, Loss=3.9700379371643066
Step 100, Loss=3.5526018142700195
Step 200, Loss=3.1110730171203613
Step 300, Loss=3.1790900230407715
Step 400, Loss=3.2271814346313477
Step 500, Loss=3.009063243865967
Step 600, Loss=2.682675838470459
Step 700, Loss=2.8985464572906494
Step 800, Loss=2.739187717437744
Step 900, Loss=2.526944637298584


In [46]:
# Testing
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

import random

def generate_text(length, diversity):
    # Get random starting text
    start_index = 0#random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = sess.run(tf.squeeze(sess.run(logits_out, feed_dict = {x: x_pred})))
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [47]:
for diversity in [0.2, 0.5, 1.0, 1.2]:
  print(diversity," - diversity----------------------------------------")
  print(generate_text(300, diversity))
  print('******************************************************')

0.2  - diversity----------------------------------------
First Citizen:
Before we proceed any furtathe ted  iin te te te ne wis ralle cirine ten on tite rite thito te  te be te ine to retill
lIl

Ficte wizen
U it tes ibn ter hinnt te shor teal in tite bte te ise yon inre ar iter Ne tie se ciren ire tin in wet al tis to to es tare ante Fenllllt ant Ciinres y in ten
Fon tes iine wen
Fhi tenle te 
******************************************************
0.5  - diversity----------------------------------------
First Citizen:
Before we proceed any fure ante. hesonribe tee . I oto vicin: eass thitt aan, ery itel he harat on iear Fas wito te ta, teel itenll hhitu wiandheer citilzt tar iiarnte u. hom cponf tae tou we ovs te ten ars
Itlkhe cint iisnyitt
I oon ret icenr taEva Nr the tiis rten bit  wete geoves anrlder ihztt et gk uhhe tato u ciner ChE
******************************************************
1.0  - diversity----------------------------------------
First Citizen:
Before we proceed any 